In [1]:
import pandas as pd
import seaborn as sns
import pycaret as pyc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss # use as custom input
from pycaret.classification import *
import numpy as np
le = LabelEncoder()

CLASSIFICATION_TARGET = 'h1n1_vaccine'

data = pd.read_csv('../../data/training_set_features.csv', index_col=0)
target = pd.read_csv('../../data/training_set_labels.csv')[CLASSIFICATION_TARGET]
df = data.join(target, how='inner')
# tdata.colurain, test = train_test_split(df, shuffle=False)data.columns
df = df.astype('category')
for col in df:
    df[col] = df[col].cat.codes
    
cols = df.drop(CLASSIFICATION_TARGET, axis=1).columns

In [2]:
clf = setup(data=df, target=CLASSIFICATION_TARGET, session_id=123, normalize=True, fix_imbalance=True, html=True, feature_selection=True,
            remove_outliers=True, remove_multicollinearity=True, remove_perfect_collinearity=True, multicollinearity_threshold=0.8,
           log_profile=True, fold_strategy='stratifiedkfold')

add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False)

,Description,Value
0,session_id,123
1,Target,h1n1_vaccine
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(26707, 36)"
5,Missing Values,False
6,Numeric Features,35
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


Name                                                        LogLoss
Display Name                                                LogLoss
Score Function                <function log_loss at 0x7f7df56091e0>
Scorer               make_scorer(log_loss, greater_is_better=False)
Target                                                         pred
Args                                                             {}
Greater is Better                                             False
Multiclass                                                     True
Custom                                                         True
Name: logloss, dtype: object

### Fitting PyCaret models

In [3]:
np.random.seed(1)

top10 = compare_models(n_select = 5, sort='AUC') 
# tune top 5 base models
tuned_top10 = [tune_model(i, optimize='AUC', search_library='optuna') for i in top10]
# ensemble top 5 tuned models
# bagged_top5 = [ensemble_model(i) for i in tuned_top10]
# blend top 5 base models 
#blender = blend_models(estimator_list = bagged_top5) 
# select best model tprin
best = automl(optimize = 'AUC')
tuned_top10

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.8080,0.8496,0.7509,0.8258,0.8149,0.4624,0.4666,6.6317
1,0.8136,0.8149,0.7364,0.8212,0.8170,0.4551,0.4560,6.4372
2,0.8243,0.8453,0.7632,0.8358,0.8290,0.4966,0.4989,6.0677
3,0.8024,0.8324,0.7383,0.8187,0.8089,0.4426,0.4459,6.8262
4,0.8046,0.8231,0.7377,0.8190,0.8105,0.4448,0.4475,6.7484
5,0.8198,0.8549,0.7667,0.8358,0.8260,0.4941,0.4981,6.2233
6,0.8046,0.8305,0.7452,0.8221,0.8115,0.4529,0.4568,6.7484
7,0.8125,0.8395,0.7452,0.8243,0.8174,0.4631,0.4652,6.4761
8,0.8176,0.8341,0.7514,0.8285,0.8221,0.4762,0.4781,6.3011
9,0.7932,0.8102,0.7136,0.8051,0.7983,0.4049,0.4065,7.1414


[LGBMClassifier(bagging_fraction=0.44004050758393054, bagging_freq=7,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.9513729201525046, importance_type='split',
                learning_rate=0.0029473190862944706, max_depth=-1,
                min_child_samples=77, min_child_weight=0.001,
                min_data_in_leaf=433, min_split_gain=0.2551873540082712,
                n_estimators=283, n_jobs=-1, num_leaves=119, objective=None,
                random_state=123, reg_alpha=6.01737607685764e-10,
                reg_lambda=1.8532035548196684e-06, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0),
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                        class_weight='balanced_subsample', criterion='gini',
                        max_depth=8, max_features=0.5789322302831629,
                        max_leaf_nodes=None, max_samples=None,
                       

In [17]:
top10[-1].save_model('h1_model')